<a href="https://colab.research.google.com/github/javmohir/malumotlarga_ishlov_berish/blob/main/SQL2-qism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import sqlite3 as sql

Yangi ma'lumotlar omborini yaratamiz

In [ ]:
db=sql.connect("newdb.db")
cursor=db.cursor()

Ishlash uchun onlayndan jadval yuklab olamiz

In [ ]:
df=pd.read_csv("https://raw.githubusercontent.com/anvarnarz/praktikum_datasets/main/Electricity_Production_By_Source.csv")
df

DFni omborga yuklashdan avval, kelin ustunlar nomini qisqartiraylik. Bunday qilishimizga sabab, keyinchalik SQL komandalar yozganda uzun nomlarni yozish o'ziga yarasha noqulayliklar keltirib chiqaradi.



In [ ]:
cols=['country','code','year','coal','gas','hydro','renewable_other','solar','oil','wind','nuclear']
df.columns=cols
df.head()

.to_sql() metodi yordamida jadvalni to'g'ridan to'g'ri omborga ko'chirishimiz mumkin.

Jadvalga jadval deb yangi nom beramiz.



In [ ]:
df.to_sql('jadval',db)

Jadvalni qaytib o'qiymiz

In [ ]:
query="SELECT * FROM jadval"
df2=pd.read_sql(query, db, index_col='index')

In [ ]:
df2.head()

In [ ]:
df2.info()

In [ ]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

In [ ]:
cursor.execute("PRAGMA table_info(jadval);")
cursor.fetchall()

In [ ]:
query="SELECT * FROM jadval WHERE country='Uzbekistan';"
dfuz=pd.read_sql(query, db, index_col='index')
dfuz

In [ ]:
dfuz.sort_values(by='year', ascending=False)

Omborda bo'sh jadval yaratish
Jadval yaratish qadamlari:

CREATE TABLE - jadval yaratish buyrug'i
IF NOT EXISTS - jadval mavjud bo'lmasa
worldpop - jadval nomi
Qavs ichida jadval ustunlari, ustundagi ma'lumot turi va qo'shimcha parametrlar vergul bilan ajratib yoziladi.
Quyidagi misolada jadvalimiz 5 ta ustundan iborat:

rank - davlat reytingi (zichlik bo'yicha),
INT ya'ni butun son.
country - davlat nomi,
 TEXT - matn,
 PRIMARY KEY - birlamchi qiymat, qatorga shu ustun qiymati bilan murojaat qilinadi (takrorlanman bo'lishi kerak)
density - zichlik,
 DOUBLE - o'nlik son,
population - aholi soni,
 NOT NULL - bo'sh bo'lmasligi (qiymat berilishi shart)
area - maydoni,
INT - butun son,
 NOT NULL - qiymat berilishi shart.

In [ ]:
query="""
CREATE TABLE IF NOT EXISTS worldpop(
    rank INT,
    country TEXT PRIMARY KEY,
    dinsity DOUBLE,
    population DOUBLE NOT NULL,
    area INT NOT NULL
);
"""
cursor.execute(query)
db.commit()

In [ ]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

In [ ]:
cursor.execute("PRAGMA table_info(worldpop);")
cursor.fetchall()

In [ ]:
query="SELECT * FROM worldpop"
df4=pd.read_sql(query, db)
df4

Jadvalga yangi ma'lumot qo'shish
SQL jadvlaga ushbu manzilda joylashgan ma'lumotlarni kiritamiz: https://github.com/anvarnarz/praktikum_datasets/blob/main/world_population.csv

In [ ]:
query="INSERT INTO worldpop VALUES (1, 'Macau', 21946.4667, 658.3940, 30)"
cursor.execute(query)
db.commit()

In [ ]:
cursor.execute("SELECT * FROM worldpop")
cursor.fetchall()

In [ ]:
pd.read_sql("SELECT * FROM worldpop", db, index_col='rank')

In [ ]:
query = """
INSERT INTO worldpop (country,population,area)
VALUES ('Uzbekistan', 33935.7630, 447400)
"""
cursor.execute(query)
db.commit()

In [ ]:
query = 'UPDATE worldpop SET rank=133, dinsity=75.8511 WHERE country="Uzbekistan"'
cursor.execute(query)
db.commit()

In [ ]:
pd.read_sql("SELECT * FROM worldpop", db)

Jadvaldan ma'lumot o'chirish

In [ ]:
query = 'DELETE FROM worldpop WHERE country="Macau"'
cursor.execute(query)
db.commit()

In [ ]:
query='DELETE FROM worldpop WHERE country="Uzbekistan"'
cursor.execute(query)
db.commit()

tekshirib ko'ramiz.

In [ ]:
pd.read_sql("select * from worldpop", db)

Jadvalni butunlay o'chirib tashlash

In [ ]:
cursor.execute("DROP TABLE worldpop")
db.commit()

In [ ]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
cursor.fetchall()

SQL buyruqlarni f-string yordamida yozish
Biz yuqorida SQL buyruqlarni oddiy matn ko'rinishida yozayapamiz, demak Pythondagi f-string yordamida matnlarga o'zgaruvchilarning qiymatini ham qo'shish mumkin.

Keling boshlanishiga quyidagi DF ni yaratib olamiz, so'ngra DF qatorlarini yangi SQL jadvalga f-string yordamida yozamiz.

In [ ]:
df = pd.read_csv("https://github.com/anvarnarz/praktikum_datasets/raw/main/world_population.csv", index_col='rank')
df.head()

In [ ]:
query="""
CREATE TABLE IF NOT EXISTS worldpop(
rank INT,
country TEXT PRIMARY KEY,
density DOUBLE,
population DOUBLE NOT NULL,
area INT NOT NULL
);
"""
cursor.execute(query)
db.commit()

In [ ]:
uz=df[df.country=="Uzbekistan"]
uz

In [ ]:
rank = uz.index.values[0]
country = uz.at[rank, 'country']
density = uz.at[rank, 'density']
population = uz.at[rank, 'pop2021']
area = uz.at[rank, 'area']
print(f"{rank=}, {country=}, {density=}, {population=}, {area=}")

In [ ]:
query=f"""
INSERT INTO worldpop VALUES({rank}, '{country}', {density}, {population}, {area})
"""
cursor.execute(query)
db.commit()

In [ ]:
query="SELECT * FROM worldpop"
cursor.execute(query)
cursor.fetchall()

In [ ]:
db.close()